In [21]:
!pip install backtesting ta-lib seaborn matplotlib

In [22]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
from backtesting.test import GOOG
import talib as ta
import seaborn as sns
import matplotlib.pyplot as plt

print(GOOG)

              Open    High     Low   Close    Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900
2004-08-20  101.01  109.08  100.50  108.31  11428600
2004-08-23  110.75  113.48  109.05  109.40   9137200
2004-08-24  111.24  111.60  103.57  104.87   7631300
2004-08-25  104.96  108.00  103.88  106.00   4598900
...            ...     ...     ...     ...       ...
2013-02-25  802.30  808.41  790.49  790.77   2303900
2013-02-26  795.00  795.95  784.40  790.13   2202500
2013-02-27  794.80  804.75  791.11  799.78   2026100
2013-02-28  801.10  806.99  801.03  801.20   2265800
2013-03-01  797.80  807.14  796.15  806.19   2175400

[2148 rows x 5 columns]


In [23]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_windows = 14

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

#stats = bt.run()
stats, heatmap = bt.optimize(
    upper_bound=range(50,85,5),
    lower_bound=range(10,45,5),
    rsi_windows=range(10,45,5),
    #maximize=optim_func,
    maximize='Sharpe Ratio',
    constraint= lambda param: param.upper_bound > param.lower_bound,
    return_heatmap=True
    #max_tries=100
)

#print(stats)

#hm = heatmap.groupby(["upper_bound","lower_bound"]).mean().unstack()
#sns.heatmap(hm)
#plt.show()
plot_heatmaps(heatmap, agg="mean")

#print(hm)
#bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 343 configurations.
  output = _optimize_grid()


GridPlot(id='p16186', ...)

# bt.run()
Start                     2004-08-19 00:00:00\
End                       2013-03-01 00:00:00\
Duration                   3116 days 00:00:00\
Exposure Time [%]                   29.329609\
Equity Final [$]                      15677.5\
Equity Peak [$]                       15835.7\
Return [%]                             56.775\
Buy & Hold Return [%]              703.458242\
Return (Ann.) [%]                    5.416737\
Volatility (Ann.) [%]               23.502544\
Sharpe Ratio                         0.230475\
Sortino Ratio                        0.386423\
Calmar Ratio                         0.110499\
Max. Drawdown [%]                  -49.020734\
Avg. Drawdown [%]                   -6.135224\
Max. Drawdown Duration     1603 days 00:00:00\
Avg. Drawdown Duration       94 days 00:00:00\
Trades                                   10\
Win Rate [%]                             80.0\
Best Trade [%]                      21.397983\
Worst Trade [%]                    -20.449092\
Avg. Trade [%]                       5.357299\
Max. Trade Duration         202 days 00:00:00\
Avg. Trade Duration          98 days 00:00:00\
Profit Factor                        3.862903\
Expectancy [%]                       5.995809\
SQN                                  1.227626\
_strategy                       RsiOscillator\
_equity_curve                          Equ...\
_trades                      Size  EntryBa...\
dtype: object

# bt.optimize(upper_bound = range(50,85,5),lower_bound = range(10,45,5),rsi_windows = range(10,30,2),maximize='Sharpe Ratio')
Start                     2004-08-19 00:00:00\
End                       2013-03-01 00:00:00\
Duration                   3116 days 00:00:00\
Exposure Time [%]                   62.662942\
Equity Final [$]                     45486.35\
Equity Peak [$]                      45573.15\
Return [%]                           354.8635\
Buy & Hold Return [%]              703.458242\
Return (Ann.) [%]                   19.448742\
Volatility (Ann.) [%]               32.629176\
Sharpe Ratio                         0.596054\
Sortino Ratio                        1.122836\
Calmar Ratio                          0.35036\
Max. Drawdown [%]                  -55.510789\
Avg. Drawdown [%]                   -4.037504\
Max. Drawdown Duration      548 days 00:00:00\
Avg. Drawdown Duration       34 days 00:00:00\
Trades                                   17\
Win Rate [%]                        76.470588\
Best Trade [%]                      41.689994\
Worst Trade [%]                    -27.637707\
Avg. Trade [%]                      11.346119\
Max. Trade Duration         240 days 00:00:00\
Avg. Trade Duration         119 days 00:00:00\
Profit Factor                        5.778866\
Expectancy [%]                      12.740143\
SQN                                  2.525661\
_strategy                 RsiOscillator(up...\
_equity_curve                           Eq...\
_trades                       Size  EntryB...\
dtype: object

In [24]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_windows = 14

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):
        if (crossover(self.daily_rsi, self.upper_bound) and self.weekly_rsi[-1]>self.upper_bound):
            self.position.close()
        elif (crossover(self.lower_bound, self.daily_rsi) and self.weekly_rsi[-1]<self.lower_bound):# and self.daily_rsi[-1]<self.weekly_rsi[-1]
            self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
[182121:182121:0100/000000.138610:ERROR:zygote_linux.cc(675)] write: Broken pipe (32)


Abriendo en una sesión existente del navegador


GridPlot(id='p17049', ...)

longs y shorts

In [25]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 75
    lower_bound = 35
    rsi_windows = 12

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):
        if crossover(self.daily_rsi, self.upper_bound):
            if self.position.is_long:
                #print(self.position.size)
                #print(self.position.pl_pct)
                self.position.close()
                self.sell()

        elif crossover(self.lower_bound, self.daily_rsi):
            if self.position.is_short or not self.position:
                self.position.close()
                self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Abriendo en una sesión existente del navegador


GridPlot(id='p18304', ...)

# Gestion de riesgo

stop-loss y take-profit

In [26]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 75
    lower_bound = 35
    rsi_windows = 12

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):

        price = self.data.Close[-1]

        if crossover(self.daily_rsi, self.upper_bound):
            self.position.close()

        elif crossover(self.lower_bound, self.daily_rsi):
            self.buy(tp=1.15*price,sl=0.95*price)

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Abriendo en una sesión existente del navegador


GridPlot(id='p19559', ...)

tamaño de la orden

In [27]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 75
    lower_bound = 35
    rsi_windows = 12

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):

        price = self.data.Close[-1]

        if crossover(self.daily_rsi, self.upper_bound):
            self.position.close()

        elif self.lower_bound > self.daily_rsi[-1]:
            #self.buy(size=1)
            self.buy(size=0.02)

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
[182435:182435:0100/000000.985506:ERROR:zygote_linux.cc(675)] write: Broken pipe (32)


Abriendo en una sesión existente del navegador


GridPlot(id='p20814', ...)

funcion barssince()

In [32]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_windows = 14

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

    def next(self):

        price = self.data.Close[-1]

        if (self.daily_rsi[-1] > self.upper_bound and barssince(self.daily_rsi < self.upper_bound==3)):
            self.position.close()

        elif self.lower_bound > self.daily_rsi[-1]:
            #self.buy(size=1)
            self.buy(size=0.02)

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()

print(stats['_trades'].to_string())
bt.plot()

Empty DataFrame
Columns: [Size, EntryBar, ExitBar, EntryPrice, ExitPrice, PnL, ReturnPct, EntryTime, ExitTime, Duration]
Index: []


/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p24337', ...)

Abriendo en una sesión existente del navegador


[183101:183101:0100/000000.509341:ERROR:zygote_linux.cc(675)] write: Broken pipe (32)
